# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [115]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/basharkhaddage/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/basharkhaddage/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [116]:
# Create a database engine
engine = create_engine('sqlite:///disaster_response_pipeline.db')

# Load data from the database into a DataFrame
query = 'SELECT * FROM full_dataset'
df = pd.read_sql(query, engine)

# Display the loaded data
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [231]:
df.shape

(26216, 40)

In [233]:
rows_with_twos = df.isin([2]).any(axis=1)
filtered_df = df[~rows_with_twos]
filtered_df.shape

(26027, 40)

In [234]:
filtered_df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [237]:
# X represents features  
# y represents labels

engine = create_engine('sqlite:///disaster_response_pipeline.db')
X = filtered_df['message']
y = filtered_df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [238]:
def tokenize(text):
    
    """
    Tokenizes message data
    
    INPUT:
       text (string): message text data
    
    OUTPUT:
        (DataFrame) clean_messages: array of tokenized message data
    """
        
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)     # Remove non-alphanumeric characters using regex
    words = word_tokenize(text)                    # Tokenize the text into words
    words = [word.lower() for word in words]       # Convert words to lowercase
    stop_words = set(stopwords.words('english'))   # Remove stopwords
    words = [word for word in words if word not in stop_words]
    stemmer = PorterStemmer()                      # Initialize Porter Stemmer
    words = [stemmer.stem(word) for word in words] # Apply stemming
    
    return words


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [239]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)
pipe.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fde19b31f70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [242]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
21047,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12462,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
25670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13913,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
14720,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2895,1,1,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20602,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [243]:
X_train.shape, X_test.shape

((18218,), (7809,))

In [244]:
y_train.shape , y_test.shape

((18218, 36), (7809, 36))

In [274]:
from sklearn.metrics import classification_report
y_test_pred = pipe.predict(X_test)      
y_test_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [275]:
print(classification_report(y_test, y_test_pred, target_names=y_test.columns))


                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5988
               request       0.83      0.47      0.60      1356
                 offer       0.00      0.00      0.00        43
           aid_related       0.77      0.69      0.72      3225
          medical_help       0.73      0.08      0.14       599
      medical_products       0.81      0.05      0.09       369
     search_and_rescue       0.79      0.05      0.10       216
              security       0.50      0.01      0.01       148
              military       0.71      0.04      0.08       249
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.40      0.54       474
                  food       0.85      0.63      0.72       877
               shelter       0.84      0.33      0.47       684
              clothing       0.88      0.11      0.20       123
                 money       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [287]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'clf__n_estimators': [5]
}

cv = GridSearchCV(pipe, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [288]:
cv.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fde19b31f70>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'vect__max_features': [1000, 5000]})

In [289]:
y_pred2=cv.predict(X_test)

In [290]:
print(classification_report(y_test, y_pred2)) 

              precision    recall  f1-score   support

           0       0.85      0.91      0.88      5988
           1       0.81      0.50      0.62      1356
           2       0.00      0.00      0.00        43
           3       0.74      0.70      0.72      3225
           4       0.55      0.17      0.25       599
           5       0.75      0.16      0.27       369
           6       0.77      0.14      0.24       216
           7       0.00      0.00      0.00       148
           8       0.57      0.05      0.09       249
           9       0.00      0.00      0.00         0
          10       0.79      0.51      0.62       474
          11       0.83      0.71      0.76       877
          12       0.81      0.50      0.62       684
          13       0.76      0.26      0.39       123
          14       0.62      0.05      0.10       182
          15       0.50      0.03      0.06        65
          16       0.65      0.14      0.23       259
          17       0.81    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [306]:
#Adaboost Classifier
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import AdaBoostClassifier

pipe2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

pipe2.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('best', TruncatedSVD()),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [307]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('best', TruncatedSVD()),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [308]:
y_test_pred_ada = pipe2.predict(X_test)
print(classification_report(y_test_pred_ada, y_test))

              precision    recall  f1-score   support

           0       1.00      0.77      0.87      7788
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         0
           3       0.04      0.47      0.07       256
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         3
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         1
          17       0.00    

### 9. Export your model as a pickle file

In [310]:
# Save the 'pipe_cv' pipeline to a pickle file using pickle

model_filename = 'best_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(cv, model_file)

print(f"Best model saved as {model_filename}")


Best model saved as best_model.pkl


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.